In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns',100)

import xgboost as xg
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer,IterativeImputer
from sklearn.model_selection import RepeatedKFold,cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

ab=pd.DataFrame(test['Id'])
target=train['SalePrice']
train.shape

In [ ]:
test

In [ ]:
#Missing value
train_null=[i for i in train.columns if train[i].isnull().sum()>500]    #dropping columns with null more than 500
test_null=[i for i in test.columns if test[i].isnull().sum()>500]
train.drop(train_null,axis=1,inplace=True)
test.drop(test_null,axis=1,inplace=True)

train.drop('SalePrice',axis=1,inplace=True)


In [ ]:
train_cat=train.select_dtypes(include='object')
train['MSZoning']=train['MSZoning'].map({'FV':4,'RL':3,'RH':2,'RM':1,'C (all)':0})
train['Street']=train['Street'].map({'Grvl':0,'Pave':1})
train['LandContour']=train['LandContour'].map({'HLS':3,'Low':3,'Lvl':2,'Bnk':1})
train['Utilities']=train['Utilities'].map({'AllPub':1,'NoSeWa':0})
train['LotConfig']=train['LotConfig'].map({'CullDSac':4,'FR3':3,'FR2':2,'Corner':1,'Inside':0})
train['MasVnrType']=train['MasVnrType'].map({'Stone':3,'BrkFace':2,'BrkCmn':1,'None':0})
train['ExterQual']=train['ExterQual'].map({'Ex':3,'Fa':0,'Gd':2,'TA':0})
train['ExterCond']=train['ExterCond'].map({'Ex':3,'Fa':0,'Gd':2,'TA':0})
train['Foundation']=train['Foundation'].map({'BrkTil':3,'PConc':5,'Wood':4,'Stone':2,'Slab':0,'CBlock':1})
train['BsmtCond']=train['BsmtCond'].map({'Gd':3,'TA':0,'Fa':2,'Po':0})
train['Heating']=train['Heating'].map({'GasA':4,'GasW':3,'OthW':2,'Floor':1,'Grav':0,'Wall':1})
train['HeatingQC']=train['HeatingQC'].map({'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0})
train['Electrical']=train['Electrical'].map({'SBrkr':4,'FuseA':3,'FuseF':2,'FuseP':1,'Mix':0})
train['KitchenQual']=train['KitchenQual'].map({'Ex':4,'Gd':3,'TA':2,'Fa':1})
train['GarageFinish']=train['GarageFinish'].map({'Fin':2,'RFn':1,'Unf':0})
train['GarageQual']=train['GarageQual'].map({'Gd':4,'TA':3,'Ex':2,'Fa':1,'Po':0})
train['GarageCond']=train['GarageCond'].map({'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0})
train['PavedDrive']=train['PavedDrive'].map({'Y':2,'P':1,'N':0})


test_cat=test.select_dtypes(include='object')
test['MSZoning']=test['MSZoning'].map({'FV':4,'RL':3,'RH':2,'RM':1,'C (all)':0})
test['Street']=test['Street'].map({'Grvl':0,'Pave':1})
test['LandContour']=test['LandContour'].map({'HLS':3,'Low':3,'Lvl':2,'Bnk':1})
test['Utilities']=test['Utilities'].map({'AllPub':1,'NoSeWa':0})
test['LotConfig']=test['LotConfig'].map({'CullDSac':4,'FR3':3,'FR2':2,'Corner':1,'Inside':0})
test['MasVnrType']=test['MasVnrType'].map({'Stone':3,'BrkFace':2,'BrkCmn':1,'None':0})
test['ExterQual']=test['ExterQual'].map({'Ex':3,'Fa':0,'Gd':2,'TA':0})
test['ExterCond']=test['ExterCond'].map({'Ex':3,'Fa':0,'Gd':2,'TA':0})
test['Foundation']=test['Foundation'].map({'BrkTil':3,'PConc':5,'Wood':4,'Stone':2,'Slab':0,'CBlock':1})
test['BsmtCond']=test['BsmtCond'].map({'Gd':3,'TA':0,'Fa':2,'Po':0})
test['Heating']=test['Heating'].map({'GasA':4,'GasW':3,'OthW':2,'Floor':1,'Grav':0,'Wall':1})
test['HeatingQC']=test['HeatingQC'].map({'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0})
test['Electrical']=test['Electrical'].map({'SBrkr':4,'FuseA':3,'FuseF':2,'FuseP':1,'Mix':0})
test['KitchenQual']=test['KitchenQual'].map({'Ex':4,'Gd':3,'TA':2,'Fa':1})
test['GarageFinish']=test['GarageFinish'].map({'Fin':2,'RFn':1,'Unf':0})
test['GarageQual']=test['GarageQual'].map({'Gd':4,'TA':3,'Ex':2,'Fa':1,'Po':0})
test['GarageCond']=test['GarageCond'].map({'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0})
test['PavedDrive']=test['PavedDrive'].map({'Y':2,'P':1,'N':0})




In [ ]:
test=pd.get_dummies(test,columns=["SaleType",'LotShape','Functional','BsmtQual','SaleCondition','LandSlope','CentralAir','Neighborhood','Condition1','GarageType',"BsmtFinType2",'RoofStyle'],drop_first=True)
train=pd.get_dummies(train,columns=["SaleType",'LotShape','Functional','BsmtQual','SaleCondition','LandSlope','CentralAir','Neighborhood','Condition1','GarageType',"BsmtFinType2",'RoofStyle'],drop_first=True)


In [ ]:
print(train.shape,test.shape)
a=['Condition2','BsmtExposure','BsmtFinType1','BldgType','RoofMatl','Exterior1st','Exterior2nd']
train['BsmtExposure']=train['BsmtExposure'].map({'No':3,'Av':2,'Gd':1,'Mn':0})
test['BsmtExposure']=test['BsmtExposure'].map({'No':3,'Av':2,'Gd':1,'Mn':0})


train['BsmtFinType1']=train['BsmtFinType1'].map({'Unf':3,'GLQ':2,'ALQ':1,'BLQ':0,'Rec':1,'LwQ':0})
test['BsmtFinType1']=test['BsmtFinType1'].map({'Unf':3,'GLQ':2,'ALQ':1,'BLQ':0,'Rec':1,'LwQ':0})


In [ ]:
train=train.drop(['Exterior1st','Exterior2nd','Condition2','BldgType','RoofMatl','Exterior1st','Exterior2nd','HouseStyle'],axis=1)
test=test.drop(['Exterior1st','Exterior2nd','Condition2','BldgType','RoofMatl','Exterior1st','Exterior2nd','HouseStyle'],axis=1)

In [ ]:

from sklearn.svm import SVR

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.svm import SVR
from sklearn.feature_selection  import SelectKBest,chi2,mutual_info_regression,VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor


pipeline = Pipeline(steps=[('i',KNNImputer(n_neighbors=27)),('s',SelectKBest(score_func=mutual_info_regression,k=69)),('normalize', StandardScaler()), ('model',xg.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 10, seed = 123))])
# prepare the model with target scaling
model = TransformedTargetRegressor(regressor=pipeline, transformer=MinMaxScaler())
# evaluate model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, train, target, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# convert scores to positive

# summarize the result
print(np.mean(scores))



In [ ]:
s_mean

In [ ]:
model.fit(train,target)
a=model.predict(test)

In [ ]:
a

In [ ]:



ab['SalePrice']=a

In [ ]:
ab.to_csv('ver1.csv',index=False)

In [ ]:
ab

In [ ]:
ab.to_csv('ver1.csv',index=False)